In [16]:
import asf_search as asf
import geopandas as gpd
import fiona, os, shutil
from fiona.crs import to_string
from pathlib import Path

In [17]:
start = '2022-07-01T00:00:00.000Z'
end = '2022-07-10T00:23:00.000Z'

In [18]:
path = r'C:\Users\ncu\Desktop\millet_farming\shape_file\mahendragarh.shp'
df = gpd.read_file(path)
fi = fiona.open(path, crs='epsg:4326')

In [19]:
print('Total polygons: ', len(list(fi)))
print('CRS: ', to_string(fi.crs))
print("Bounds: ", fi.bounds)
print("Schema: ", fi.schema)

Total polygons:  1
CRS:  EPSG:4326
Bounds:  (75.8889999389649, 28.147859573364258, 76.38835906982428, 28.471929550171012)
Schema:  {'properties': {'ID_0': 'int:10', 'ISO': 'str:3', 'NAME_0': 'str:75', 'ID_1': 'int:10', 'NAME_1': 'str:75', 'ID_2': 'int:10', 'NAME_2': 'str:75', 'ID_3': 'int:10', 'NAME_3': 'str:75', 'TYPE_3': 'str:50', 'ENGTYPE_3': 'str:50', 'NL_NAME_3': 'str:75', 'VARNAME_3': 'str:100'}, 'geometry': 'Polygon'}


In [20]:
shp = df.to_crs(epsg=4326)
wkt = str(shp['geometry'][0])

In [21]:
findOrbit = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1A], intersectsWith=wkt, processingLevel='GRD_HD', start=start, end=end, flightDirection='Descending')
print(findOrbit)

{
  "features": [
    {
      "geometry": {
        "coordinates": [
          [
            [
              77.371658,
              26.433838
            ],
            [
              77.701714,
              27.941822
            ],
            [
              75.158936,
              28.352581
            ],
            [
              74.863678,
              26.847361
            ],
            [
              77.371658,
              26.433838
            ]
          ]
        ],
        "type": "Polygon"
      },
      "properties": {
        "beamModeType": "IW",
        "browse": [
          "https://datapool.asf.alaska.edu/BROWSE/SA/S1A_IW_GRDH_1SDV_20220705T005249_20220705T005314_043958_053F54_E3F8.jpg"
        ],
        "bytes": 1005396112,
        "centerLat": 27.3996,
        "centerLon": 76.2764,
        "faradayRotation": null,
        "fileID": "S1A_IW_GRDH_1SDV_20220705T005249_20220705T005314_043958_053F54_E3F8-GRD_HD",
        "fileName": "S1A_IW_GRDH_1SDV_2022070

In [22]:
orbit = int(findOrbit[-1].geojson()['properties']['fileName'][49:55])
print(orbit)

43958


In [23]:
rel = (orbit - 73) % 175
rel = rel + 1
print(rel)

136


In [24]:
result = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1A], intersectsWith=wkt, processingLevel=[asf.PRODUCT_TYPE.GRD_HD], start=start, end=end, flightDirection='Descending', relativeOrbit=rel)
metadata = result.geojson()

In [25]:
metadata

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'coordinates': [[[77.371658, 26.433838],
      [77.701714, 27.941822],
      [75.158936, 28.352581],
      [74.863678, 26.847361],
      [77.371658, 26.433838]]],
    'type': 'Polygon'},
   'properties': {'beamModeType': 'IW',
    'browse': ['https://datapool.asf.alaska.edu/BROWSE/SA/S1A_IW_GRDH_1SDV_20220705T005249_20220705T005314_043958_053F54_E3F8.jpg'],
    'bytes': 1005396112,
    'centerLat': 27.3996,
    'centerLon': 76.2764,
    'faradayRotation': None,
    'fileID': 'S1A_IW_GRDH_1SDV_20220705T005249_20220705T005314_043958_053F54_E3F8-GRD_HD',
    'flightDirection': 'DESCENDING',
    'groupID': 'S1A_IWDV_0501_0506_043958_136',
    'granuleType': 'SENTINEL_1A_FRAME',
    'insarStackId': None,
    'md5sum': '3cc768cab129fd50097be4b92921832d',
    'offNadirAngle': None,
    'orbit': 43958,
    'pathNumber': 136,
    'platform': 'Sentinel-1A',
    'pointingAngle': None,
    'polarization': 'VV+VH',
    

In [26]:
session = asf.ASFSession().auth_with_creds('mayankbhardwaj', 'TOPsecret27')
dpath = r'C:\Users\ncu\Desktop\millet_farming\data\commonData\downloadData\\'
result.download(path = dpath, session=session)